In [83]:
import json
import os
import re
from typing import List, Dict

import pandas as pd

In [84]:
# 사용할 엔티티 타입
ENTITY_TYPES = ["PS", "LC", "OG", "DT", "TI", "QT"]

# <|PS|>엔티티<|PS|> 같은 패턴 찾는 정규식
ENTITY_PATTERN = re.compile(
    r"<\|(PS|LC|OG|DT|TI|QT)\|>(.+?)<\|\1\|>",
    flags=re.DOTALL
)

In [85]:
def parse_tagged_sentence(tagged_text: str) -> Dict:
    """
    라벨링된 문장 하나를 받아서
    - 태그가 제거된 실제 문장 text
    - 엔티티 스팬 리스트 [{"start": int, "end": int, "label": "PS"}, ...]
    를 반환합니다.
    """
    entities = []
    clean_text_parts = []
    curr_pos = 0  # clean_text 상의 현재 길이

    last_end = 0
    for match in ENTITY_PATTERN.finditer(tagged_text):
        label = match.group(1)
        surface = match.group(2)

        # 태그 이전의 일반 텍스트 추가
        before = tagged_text[last_end:match.start()]
        if before:
            clean_text_parts.append(before)
            curr_pos += len(before)

        # 엔티티 텍스트 추가
        ent_start = curr_pos
        clean_text_parts.append(surface)
        curr_pos += len(surface)
        ent_end = curr_pos

        entities.append({
            "start": ent_start,
            "end": ent_end,
            "label": label
        })

        last_end = match.end()

    # 마지막 태그 뒤에 남은 텍스트
    tail = tagged_text[last_end:]
    if tail:
        clean_text_parts.append(tail)
        curr_pos += len(tail)

    clean_text = "".join(clean_text_parts)

    return {
        "text": clean_text,
        "entities": entities
    }

In [86]:
# ---- 예시 테스트 ----
sample = """“반갑다. 힘찬 호랑이 해야” <|DT|>2022년<|DT|> 임인년(壬寅年) 첫날이 밝았다."""
parsed = parse_tagged_sentence(sample)
print(parsed["text"])
print(parsed["entities"])

for ent in parsed['entities']:
    print(parsed['text'][ent['start']:ent['end']])

“반갑다. 힘찬 호랑이 해야” 2022년 임인년(壬寅年) 첫날이 밝았다.
[{'start': 17, 'end': 22, 'label': 'DT'}]
2022년


# 1. Process data

In [87]:
dataset_name = "NIKL_NEWSPAPER_2023_CSV"
fname = "NEWSPAPER_2022_1"
sample_name = "sample1"

save_base_path = f"results/{dataset_name}-{fname}-{sample_name}"

In [88]:
df = pd.read_parquet(f"data/{dataset_name}-{fname}-{sample_name}.parquet")

In [89]:
df.head()

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_ids,sentence_offsets,text
47777,NIRW2300000001,NIRW2300000001.52999,노컷뉴스 2022년 기사,최범규 충북CBS 최범규 기자,노컷뉴스,20221122,사회,"지역>울산, 지역>대전, 지역>부산","[""NIRW2300000001.52999.1"", ""NIRW2300000001.529...","[[0, 34], [35, 85], [86, 121], [122, 196], [19...","맨몸으로 불난 집 뛰어들어 노인 구한 ‘슈퍼맨’, 정체 보니… 우체국 집배원이 맨몸..."
14948,NIRW2300000001,NIRW2300000001.23451,노컷뉴스 2022년 기사,경남CBS 송봉준 기자 송봉준,노컷뉴스,20220520,사회,"사회>사회일반, 문화>문화일반","[""NIRW2300000001.23451.1"", ""NIRW2300000001.234...","[[0, 29], [30, 95], [96, 221], [222, 312], [31...","거제 선자산 추락헬기 정비사, 새 생명 주고 ‘영면’ 경남 거제 선자산에서 발생한 ..."
54496,NIRW2300000001,NIRW2300000001.59045,노컷뉴스 2022년 기사,베이징=CBS노컷뉴스 안성용 특파원 안성용,노컷뉴스,20221229,사회,"국제>중국, 사회>사회일반, 국제>아시아","[""NIRW2300000001.59045.1"", ""NIRW2300000001.590...","[[0, 25], [26, 116], [117, 209], [210, 335], [...",中 화장장 포화에…“공터에서 시신 불태우겠다” 중국 대도시에서 코로나19 감염이 확...
37023,NIRW2300000001,NIRW2300000001.43319,노컷뉴스 2022년 기사,고상현 제주CBS 고상현 기자,노컷뉴스,20220925,사회,"지역>대구, 지역>제주, 지역>부산","[""NIRW2300000001.43319.1"", ""NIRW2300000001.433...","[[0, 29], [30, 74], [75, 177], [178, 283], [28...",도로 한가운데서 잠들어…영화배우 곽도원 음주운전 적발 제주에서 영화배우 곽도원(49...
15346,NIRW2300000001,NIRW2300000001.2381,노컷뉴스 2022년 기사,CBS노컷뉴스 조기선 선임기자 조기선,노컷뉴스,20220118,사회,"지역>강원, 지역>광주, 정치>선거","[""NIRW2300000001.2381.1"", ""NIRW2300000001.2381...","[[0, 40], [41, 109], [110, 176], [177, 259], [...",광주시교육감 후보들 출마 기자회견 잇따라 연기…아파트 붕괴 사고 ‘애도’ 광주 신축...


In [90]:
remove_ids = []
entities = []
texts = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    
    docid = row['doc_id']
    
    if not os.path.exists(os.path.join(save_base_path, f"response/{docid}.json")):
        remove_ids.append(docid)
        continue
    
    with open(os.path.join(save_base_path, f"generated/{docid}.json"), "r") as f:
        generated = json.load(f)
    
    text = generated['tagged_text']
    parsed = parse_tagged_sentence(text)
    row_entities = parsed['entities']
    
    texts.append(parsed['text'])
    entities.append(row_entities)

print(f"Removed Ids {len(remove_ids)}")

Removed Ids 4


In [91]:
df = df[~df.doc_id.isin(remove_ids)]
df['text']=texts
df['entities'] = entities

In [92]:
df = df.drop(labels=['sentence_offsets'], axis=1)

In [93]:
df.head()

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_ids,text,entities
47777,NIRW2300000001,NIRW2300000001.52999,노컷뉴스 2022년 기사,최범규 충북CBS 최범규 기자,노컷뉴스,20221122,사회,"지역>울산, 지역>대전, 지역>부산","[""NIRW2300000001.52999.1"", ""NIRW2300000001.529...","맨몸으로 불난 집 뛰어들어 노인 구한 ‘슈퍼맨’, 정체 보니… 우체국 집배원이 맨몸...","[{'start': 22, 'end': 25, 'label': 'PS'}, {'st..."
14948,NIRW2300000001,NIRW2300000001.23451,노컷뉴스 2022년 기사,경남CBS 송봉준 기자 송봉준,노컷뉴스,20220520,사회,"사회>사회일반, 문화>문화일반","[""NIRW2300000001.23451.1"", ""NIRW2300000001.234...","경남 거제 선자산 추락헬기 정비사, 새 생명 주고 ‘영면’ 경남 거제 선자산에서 발...","[{'start': 0, 'end': 9, 'label': 'LC'}, {'star..."
54496,NIRW2300000001,NIRW2300000001.59045,노컷뉴스 2022년 기사,베이징=CBS노컷뉴스 안성용 특파원 안성용,노컷뉴스,20221229,사회,"국제>중국, 사회>사회일반, 국제>아시아","[""NIRW2300000001.59045.1"", ""NIRW2300000001.590...",중 화장장 포화에…“공터에서 시신 불태우겠다” 중국 대도시에서 코로나19 감염이 확...,"[{'start': 51, 'end': 54, 'label': 'LC'}, {'st..."
37023,NIRW2300000001,NIRW2300000001.43319,노컷뉴스 2022년 기사,고상현 제주CBS 고상현 기자,노컷뉴스,20220925,사회,"지역>대구, 지역>제주, 지역>부산","[""NIRW2300000001.43319.1"", ""NIRW2300000001.433...",곽도원 음주운전 적발 제주에서 영화배우 곽도원(49·본명 곽병규)이 음주운전을 하다...,"[{'start': 0, 'end': 3, 'label': 'PS'}, {'star..."
15346,NIRW2300000001,NIRW2300000001.2381,노컷뉴스 2022년 기사,CBS노컷뉴스 조기선 선임기자 조기선,노컷뉴스,20220118,사회,"지역>강원, 지역>광주, 정치>선거","[""NIRW2300000001.2381.1"", ""NIRW2300000001.2381...",광주시교육감 후보들 출마 기자회견 잇따라 연기…아파트 붕괴 사고 ‘애도’ 광주 신축...,"[{'start': 0, 'end': 6, 'label': 'OG'}, {'star..."


# 2. Save to disk as datasets

In [94]:
from datasets import Dataset

In [95]:
# raw_dataset = Dataset.from_list([parsed, parsed])
ds = Dataset.from_pandas(df)

In [96]:
ds = ds.remove_columns(["__index_level_0__"])

In [97]:
ds

Dataset({
    features: ['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic', 'original_topic', 'sentence_ids', 'text', 'entities'],
    num_rows: 14990
})

In [98]:
ds[0]

{'file_id': 'NIRW2300000001',
 'doc_id': 'NIRW2300000001.52999',
 'title': '노컷뉴스 2022년 기사',
 'author': '최범규 충북CBS 최범규 기자',
 'publisher': '노컷뉴스',
 'date': 20221122,
 'topic': '사회',
 'original_topic': '지역>울산, 지역>대전, 지역>부산',
 'sentence_ids': '["NIRW2300000001.52999.1", "NIRW2300000001.52999.2", "NIRW2300000001.52999.3", "NIRW2300000001.52999.4", "NIRW2300000001.52999.5", "NIRW2300000001.52999.6", "NIRW2300000001.52999.7", "NIRW2300000001.52999.8", "NIRW2300000001.52999.9", "NIRW2300000001.52999.10", "NIRW2300000001.52999.11"]',
 'text': '맨몸으로 불난 집 뛰어들어 노인 구한 ‘슈퍼맨’, 정체 보니… 우체국 집배원이 맨몸으로 주택 화재 현장에 뛰어들어 70대 집주인을 구한 사실이 알려졌다. 주인공은 충북 영동군 영동우체국에서 근무하는 유지하(33)씨다. 유씨는 지난 21일 오전 11시쯤 영동읍 계산리 일대에서 우편물을 배달하던 중 한 주택에서 많은 연기가 치솟는 것을 목격했다. 급박한 상황을 직감한 유씨는 곧장 집으로 뛰어들어갔다. 집주인 70대 A씨가 평소 집에 머물고 있던 것을 알고 있던 유씨는 가장 먼저 A씨를 찾았다. 그러면서 집 뒤편에 있던 소화기로 진화하려다 쓰러져 있는 A씨를 발견했다. 유씨는 A씨를 업고 밖으로 나와 응급조치를 하며 119에 신고했다. 또 주변에 주차된 차량 탓에 소방차가 진입하기 어려울 것을 알고, 분주하게 움직이며 이동 주차도 요청했다. 유씨의 발빠른 대처에 A씨는 소중한 목숨을 구했고, 화재도 신속하게 진화됐다

In [100]:
sample = ds[0]

text = sample['text']
for ent in sample['entities']:
    print(ent['label'], text[ent['start']:ent['end']])

PS 슈퍼맨
OG 우체국
QT 70대
LC 영동군
OG 영동우체국
PS 유지하
QT 33
DT 21일
TI 오전 11시
LC 영동읍
LC 계산리
QT 70대
PS A씨
PS A씨
PS A씨
PS A씨
QT 119
PS A씨


In [103]:
dataset_dir = f"../dataset/{dataset_name}-{fname}-{sample_name}.parquet"
ds.to_parquet(dataset_dir)

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

51823991

In [102]:
# ds.save_to_disk(dataset_dir)